# Analiza danych przestrzennych - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania i oceny notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z fragmentów kodu opracowanego w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce notatnika.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu, odpowiedzi uzupełniane manualnie nie podlegają ocenie.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają na celu wyłapać podstawowe błędy. Przejście przez kod wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie i zostanie ocenione na maksymalną liczbę punktów.
- Zadania należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem). Rozmiar poziomy figur nie powinien przekraczać 20 cali.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu: IAD13
Członkowie: Zuzanna Brauer, Sandra Głowacka , Zuzanna Chmielarska

---

## Zestaw zadań 3: Elementy statystyki opisowej 3 (Notatnik 1/3)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
import zipfile

import warnings
warnings.filterwarnings("ignore")

### Zadanie 1: Przygotowanie danych [10 pkt]

#### a) Natężenie ruchu rowerowego 

Plik raw_bike_traffic_data.csv zawiera dane na temat ruchu rowerowego na moście Fremont Bridge w Seattle pobrane ze strony
https://data.seattle.gov/Transportation/Fremont-Bridge-Bicycle-Counter/65db-xm6k.

Zaimportuj zawarte w pliku dane do notatnika i utwórz na ich podstawie tablicę `bike_traffic_data` wykonaną według następujących wytycznych:

- Tablica powinna zawierać wyłącznie następujące elementy:</br>
    > Indeks: Date</br>
    > `pandas.PeriodIndex` zawierający okresy czasu, w których dokonywane były zliczenia rowerzystów.
    
    > Kolumny: Total, East, West</br>
    > Kolumny zawierają liczbę rowerzystów przejeżdżających przez most w poszczególnych godzinach w sumie oraz z podziałem na zachodnią i wschodnią stronę mostu.
- Pozostaw w tablicy wyłącznie wiersze dotyczące 2022 roku.

Zapisz tablicę do pliku bike_traffic_data.pkl.

Wyświetl przygotowaną tablicę w taki sposób aby widoczne było po 12 pierwszych i ostatnich wierszy, a zawarte w tablicy wartości wyświetlały się bez miejsc po przecinku.

In [132]:
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.namelist
    selected_file = zip_ref.namelist()[0]
    with zip_ref.open(selected_file) as file:
        bike_traffic_data = pd.read_csv(file)
        
bike_traffic_data.Date = pd.to_datetime(bike_traffic_data.Date, format="%m/%d/%Y %I:%M:%S %p")
bike_traffic_data.set_index(pd.PeriodIndex(bike_traffic_data['Date'], freq='H'), inplace=True)
bike_traffic_data.drop("Date",axis=1,inplace=True)
bike_traffic_data.columns = ["Total","East","West"]
bike_traffic_data = bike_traffic_data[bike_traffic_data.index.year == 2022]
bike_traffic_data.to_pickle("bike_traffic_data.pkl")

display(pd.concat([bike_traffic_data.head(12), bike_traffic_data.tail(12)]).style.format(precision =0))

,Total,East,West
Date,,,
2022-01-01 00:00,0,0,0
2022-01-01 01:00,0,0,0
2022-01-01 02:00,1,0,1
2022-01-01 03:00,0,0,0
2022-01-01 04:00,0,0,0
2022-01-01 05:00,1,1,0
2022-01-01 06:00,1,1,0
2022-01-01 07:00,1,1,0
2022-01-01 08:00,3,3,0


In [47]:
### Komórka testowa
# Test 1 Kształt tablicy
assert bike_traffic_data.shape == (8760, 3)
# Test 2 Kolumny
assert len(set(list(bike_traffic_data.columns)) & set(['Total', 'East', 'West'])) == 3
# Test 3 Indeks
assert (bike_traffic_data.index.name == 'Date') == True
assert isinstance(bike_traffic_data.index, pd.PeriodIndex)
assert bike_traffic_data.index.values.min() == pd.Period('2022-01-01 00:00', 'H')
assert bike_traffic_data.index.values.max() == pd.Period('2022-12-31 23:00', 'H')
# Test 4 Zawartość kolumn
assert np.all(np.isclose(bike_traffic_data[['Total', 'East', 'West']].min().values, np.array([0, 0, 0]))) == True
assert np.all(np.isclose(bike_traffic_data[['Total', 'East', 'West']].max().values, np.array([717, 464, 635]))) == True

#### b) Pogoda

Pliki 2022_01.txt, 2022_02.txt ...  2022_12.txt zawierają miesięczne raporty pogodowe (F6 Preliminary Local Climate Data Report) ze stacji NWS Seattle/Tacoma, WA pobrane ze strony https://forecast.weather.gov/product.php?site=SEW&product=CF6&issuedby=SEW.

Wyjaśnienie wykorzystanych w raportach symboli i skrótów można znaleźć pod następującym linkiem: https://www.weather.gov/grr/climateF6explain.

Zaimportuj zawarte w plikach dane do notatnika i utwórz na ich podstawie tablicę `weather_data` wykonaną według następujących wytycznych:

- Tablica powinna zawierać wyłącznie następujące elementy:</br>
    > Indeks: Date</br>
    > `pandas.PeriodIndex` zawierający okresy czasu, których dotyczą dane z raportów pogodowych.</br>
    
    > Kolumny: 2-poziomowe nagłówki w podanej niżej kolejności (nazwa 1 poziomu: podległe jej nazwy 2 poziomu) </br>
    >> Temperature: Min, Avg, Max</br>
    >> Precipitation: Total</br>
    >> Snow: Fall, Depth</br>
    >> Wind:  Avg speed, Max speed</br>
    >> Sky: Cloud cover</br>

- Zamień oznaczenia opadów śladowych na wartość 0.
- Przelicz występujące w tablicy wartości do *C (temperatura), mm (opady), m/s (wiatr) i ułamków (zachmurzenie).

Zapisz tablicę do pliku weather_data.pkl.

Wyświetl przygotowaną tablicę w taki sposób aby widoczne było po 10 pierwszych i ostatnich wierszy, a zawarte w tablicy liczby wyświetlały się z dokładnością 2 miejsc po przecinku.

In [131]:
col_lev1 = ['Temperature','Temperature','Temperature','Precipitation','Snow','Snow','Wind','Wind','Sky']
columns = ['DY','Max','Min' ,'Avg', 'DEP', 'HDD', 'CDD' , 'Total',  'Fall', 'Depth', 'Avg speed' ,'Max speed','DIR','MIN2', 'PSBL', 'Cloud cover' ,'WX','SPD', 'DR']
col_lev2 = ['Min' ,'Avg', 'Max', 'Total',  'Fall', 'Depth', 'Avg speed' ,'Max speed','Cloud cover' ]
weather_data = pd.DataFrame()
day_in_month= [ 31,28,31,30,31,30,31,31,30,31,30,31]
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    for i in range(1, 13):
        selected_file = f'raw_weather_data/2022_{i:02d}.txt'
        
        with zip_ref.open(selected_file) as file:
            df_month = pd.read_csv(file, skiprows=18, nrows=day_in_month[i-1], sep='\s+', index_col=False, names=columns,header=None)
            df_month['Date'] = pd.to_datetime('2022' + '-' + str(i) + '-' + df_month['DY'].astype(str).str.zfill(2))
            df_month.set_index('Date', inplace=True)           
            df_month.drop(columns=['DY','DEP','HDD','CDD','DIR','MIN2','PSBL','WX','SPD','DR'], inplace=True)
            
            weather_data = pd.concat([weather_data, df_month])
weather_data = weather_data[col_lev2]
weather_data.columns = pd.MultiIndex.from_arrays([col_lev1, col_lev2])
weather_data.replace("T", 0, inplace=True)
weather_data.replace('M', np.nan, inplace=True)

weather_data.index = pd.PeriodIndex(weather_data.index, freq='D')
weather_data = weather_data.astype(np.float64)
weather_data.iloc[:,:3] = weather_data.iloc[:,:3].apply(lambda x: (x-32)*5/9) # F -> C
weather_data.iloc[:,3:6] = weather_data.iloc[:,3:7].apply(lambda x: (x*25.4)) #inch -> mm
weather_data.iloc[:,-3:-1] = weather_data.iloc[:,-3:-1].apply(lambda x: x*1609.34/3600) #mile/h -> m/s
weather_data.iloc[:,-1] = weather_data.iloc[:,-1].apply(lambda x: x/10) # na ułamek

display(pd.concat([weather_data.head(10), weather_data.tail(10)]).style.format(precision =2))
weather_data.to_pickle("weather_data.pkl")

In [130]:
### Komórka testowa
# Test 1 Kształt tablicy
assert weather_data.shape == (365, 9)
# Test 2 Kolumny
assert len(set(list(weather_data.columns)) & set([('Temperature', 'Min'), ('Temperature', 'Avg'), ('Temperature', 'Max'), ('Precipitation', 'Total'),
            ('Snow', 'Fall'), ('Snow', 'Depth'), ('Wind', 'Avg speed'), ('Wind', 'Max speed'), ('Sky', 'Cloud cover')])) == 9
# Test 3 Indeks
assert (weather_data.index.name == 'Date') == True
assert isinstance(weather_data.index, pd.PeriodIndex) == True
assert weather_data.index.values.min() == pd.Period('2022-01-01', 'D')
assert weather_data.index.values.max() == pd.Period('2022-12-31', 'D')
# Test 4 Zawartość kolumn
assert np.all(np.isclose(weather_data.min().values, np.array([-7.77777778, -5.55555556, -3.33333333, 0., 0., 0., 0.89407778, 3.12927222, 0.]))) == True
assert np.all(np.isclose(weather_data.max().values, np.array([20., 26.66666667, 35., 75.438, 66.04, 76.2, 8.31492333, 16.54043889, 1.]))) == True